## 引入工具

In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
# %watermark -p numpy,sklearn,pandas
# %watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
# %watermark -p torch,torchvision,torchaudio
# %watermark -p tensorflow,tensorboard,tflite
# %watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
# %matplotlib inline
# %config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

# from IPython.display import display, Markdown, HTML, IFrame, Image, Javascript
# from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
# display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, logging, time, random, math
import json, base64, requests, shutil
import argparse, shlex, signal
import numpy as np

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x):
    try: 
        x = x.strip()
        if x[0] == '/':
            with open(x) as fr:
                x = fr.read()
        elif 'github' in x or 'gitee' in x:
            if x.startswith('import '):
                x = x[7:]
            if x.startswith('https://'):
                x = x[8:]
            if not x.endswith('.py'):
                x = x + '.py'
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                uri = 'https://' + x
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                uri = x.replace('github.com', 'raw.githubusercontent.com')
                mod = uri.split('/')
                for s in ['main', 'master']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except:
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)


## 解析配置

In [2]:
_IMPORT('/data/nb_easy/easy_widget.py')

In [3]:
from utils import EVENTS

RACEURL = 'http://116.85.68.193:9119'
MSGKEY = 'nb.zmq.repnet_tf.gamma103'

with open('schema.json', 'r') as fr:
    schema_string = fr.read()
    schema_string = schema_string.replace('zmq.repnet_tf.test', MSGKEY)
    schema_string = schema_string.replace('http://host:port', RACEURL)
    schema = json.loads(schema_string)
easy = nbeasy_schema_parse(schema, events=EVENTS, debug=True);

Box(children=(Box(children=(VBox(children=(Tab(children=(VBox(children=(HBox(children=(Text(value='zmq.repnet_…

In [4]:
raise

RuntimeError: No active exception to reraise

In [ ]:
import numpy as np
import time
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import matplotlib.patheffects as PathEffects
%matplotlib inline

from sklearn.decomposition import PCA

In [ ]:
%%bash

DST_DIR=/data/sims/
url_prefix=https://frepai.s3.didiyunapi.com/datasets/vod/outputs
url_postfix=nb.zmq.repnet_tf.gamma103/embs_sims.npy

FILES=('20211130112549' '20211201074745' '20211201110905' '20211201161714' '20211130112549')

for file in ${FILES[@]}
do
    if [[ ! -f $DST_DIR/$file.npy ]]
    then
        wget -q -P $DST_DIR $url_prefix/$file/$url_postfix
        mv $DST_DIR/embs_sims.npy $DST_DIR/$file.npy
    fi
done

### A

In [ ]:
def plot_scatter(x, colors=None):
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], cmap=plt.cm.get_cmap('Paired'))
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('on')
    ax.axis('tight')

### PCA

In [ ]:
sims = ('20211130112549', '20211201074745', '20211201110905', '20211201161714')
pos_sims_list = []
for s in sims:
    pos_sims_list.append(np.load(f'/data/sims/{s}.npy'))
    
neg_sims = np.load(f'/data/sims/20211130112549.npy')

In [ ]:
pca = PCA(n_components=2)

In [ ]:
neg_sim.shape

In [ ]:
data = np.vstack(pos_sims_list) # np.concatenate(sims_list)

In [ ]:
il1 = np.tril_indices(64)

In [ ]:
neg_features = neg_sim[:, il1[0], il1[1]]
neg_features.shape

In [ ]:
neg_features.var(axis=1)

In [ ]:
features = data[:, il1[0], il1[1]]

In [ ]:
features[0].shape

In [ ]:
pca_pos_out = pca.fit_transform(features)
pca_pos_out.var(), pca_pos_out.mean()

In [ ]:
pca_pos_out.std(axis=1)

In [ ]:
pca_neg_out = pca.fit_transform(neg_features)
pca_neg_out.var(), pca_neg_out.mean()

In [ ]:
pca_neg_out.std(axis=1)

In [ ]:
pca.explained_variance_

In [ ]:
plot_scatter(pca_pos_out)

In [ ]:
plot_scatter(pca_neg_out)

### TSNE

In [ ]:
tsne = TSNE(random_state=0)

In [ ]:
Xt = pca.fit_transform(features)
plt.scatter(Xt[:,0], Xt[:,1])

In [ ]:
Nt = pca.transform(neg_features)
plt.scatter(Nt[:,0], Nt[:,1])

In [ ]:
pca.explained_variance_ratio_

In [ ]:
tsne_results = tsne.fit_transform(features)

In [ ]:
plot_scatter(tsne_results)

### B

In [ ]:
qg_embs1 = np.load('/data/repnet_embs_qg_1.npy')
qg_embs1.shape

In [ ]:
qg_features_1 = qg_embs1[1:10].reshape(-1, 512)
qg_features_1.shape

In [ ]:
qg_tsne_1 = tsne.fit_transform(qg_features_1)

In [ ]:
plot_scatter(qg_tsne_1)

### C

In [ ]:
mz_embs1 = np.load('/data/repnet_embs_mz_1.npy')
mz_embs1.shape

In [ ]:
mz_features_1_ = mz_embs1.reshape(-1, 512)
mz_features_1_.shape

In [ ]:
mz_tsne_1_ = tsne.fit_transform(mz_features_1_)

In [ ]:
plot_scatter(mz_tsne_1_)

In [ ]:
mz_features_1 = mz_embs1[1:10].reshape(-1, 512)
mz_features_1.shape

In [ ]:
mz_tsne_1 = tsne.fit_transform(mz_features_1)

In [ ]:
plot_scatter(mz_tsne_1)

## 视频描述